In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/realestatepriceprediction/sample_submission.csv
/kaggle/input/realestatepriceprediction/train.csv
/kaggle/input/realestatepriceprediction/test.csv


In [2]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 100
## Подгружаем нужные библиотеки, устанавливаем стили для графиков

In [3]:
train_path = '/kaggle/input/realestatepriceprediction/train.csv'
test_path = '/kaggle/input/realestatepriceprediction/test.csv'

In [4]:
df_train = pd.read_csv(train_path)
df_train.head()
## загружаем тренировочные данные, проверяем, что все загрузилось

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [5]:
df_test = pd.read_csv(test_path)
df_test.head()
## подгружаем тестовые данные и проверяем

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A


In [6]:
df_train.describe()

# Видны проблемы в следющих столбцах:
# Rooms - есть квартиры с 0 комнат, 75% от всех значений = 2 комнатам, а максимальное значение = 19 - вероятно что есть выбросы
# в LifeSquare и KitchenSquare максимальные значения выше чем Square - есть явно некорректные значения
# Floor - потенциальное место с выбросами в верхних этажах
# HouseFloor - есть дома с 0 этажей
# HouseYear - есть странные значения годов
# Остальные стоит проверить на выбросы и числовые данные, которые на самом деле являются категориями, а не числами

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [7]:
def types_df(df):
    df['Id'] = df['Id'].astype(str)
    df['DistrictId'] = df['DistrictId'].astype(str)
    return df
df_train = types_df(df_train)
df_test = types_df(df_test)

In [8]:
df_train.dtypes

Id                object
DistrictId        object
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor              int64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2         object
Ecology_3         object
Social_1           int64
Social_2           int64
Social_3           int64
Healthcare_1     float64
Helthcare_2        int64
Shops_1            int64
Shops_2           object
Price            float64
dtype: object

In [9]:
df_train.isna().sum()
## нужно будет отработать неопознанные значения в LifeSquare, Healthcare_1

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
dtype: int64

In [10]:
df_train['Rooms'].value_counts()
## Проверим количество комнат

2.0     3880
1.0     3705
3.0     2235
4.0      150
5.0       18
0.0        8
10.0       2
19.0       1
6.0        1
Name: Rooms, dtype: int64

In [11]:
def setRooms(df):
    df.loc[df['Rooms'] == 0, 'Rooms'] = 1
    df.loc[df['Rooms'] > 5, 'Rooms'] = 5
    return df

df_train = setRooms(df_train)
df_test = setRooms(df_test)
## верхние выбросы закинем в максимальное значение, которое не будем считать выбросом

In [12]:
df_train['Square'].quantile(.001), df_train['Square'].quantile(.999)
## Попробуем выкинуть все тренировочные значения, которые не попадают в этот диапазон, расширенный на 10%
## т.к. явно квартиры менее 5 кв.м. и более 200 кв.м. встреаются редко и являются выбросами

(5.128916069316165, 183.3847969007745)

In [13]:
df_train = df_train.drop(df_train[(df_train['Square'] < 5) | (df_train['Square'] > 200)].index)

In [14]:
df_train.describe()

,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,9984.000000,9984.000000,7872.000000,9984.000000,9984.000000,9984.000000,9.984000e+03,9984.000000,9984.000000,9984.000000,9984.000000,5196.000000,9984.000000,9984.000000,9984.000000
mean,1.888922,56.167854,37.035990,6.275341,8.524639,12.612680,3.993363e+03,0.118863,24.697716,5353.857772,8.035156,1142.809854,1.320012,4.231170,214068.636061
std,0.812961,18.844926,85.875020,28.581805,5.234671,6.766991,2.006609e+05,0.119063,17.532551,4006.029681,23.812438,1020.833557,1.493871,4.804695,92724.208665
min,1.000000,5.129222,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,1.000000,41.784346,22.798130,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153907.528147
50%,2.000000,52.529412,32.782779,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192290.039215
75%,2.000000,65.890819,45.112286,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249100.769116
max,5.000000,198.930182,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [15]:
## Избавимся от выбросов в KitchenSquare
## Найдем медиану KitchenSquare / Square в неошибочных значениях
def condKitch(df):
    return (df['KitchenSquare'] < 3) | (df['KitchenSquare'] >= df['Square'])
cond1 = condKitch(df_train)
medKitchen = (df_train.loc[~cond1, 'KitchenSquare'] / df_train.loc[~cond1, 'Square']).median()

In [16]:
cond1.value_counts()

False    6831
True     3153
dtype: int64

In [17]:
df_train.loc[cond1, 'KitchenSquare'] = df_train['Square'] * medKitchen
df_test.loc[condKitch(df_test), 'KitchenSquare'] = df_test['Square'] * medKitchen
## для тренировочных данных посчитаем медиану KitchenSquare / Square и в ошибочных случаях поменяем KitchenSquare на медиану от Square и в тренировочных и в тестовых данных

In [18]:
## Сделаем похожую операцию с LifeSquare предварительно заполнив "0" пустые значения
def condLife(df):
    return (df['LifeSquare'] == 0) | (df['LifeSquare'] >= df['Square'])

df_train.loc[df_train['LifeSquare'].isna(), 'LifeSquare'] = 0
df_test.loc[df_test['LifeSquare'].isna(), 'LifeSquare'] = 0

cond2 = condLife(df_train)
medLife = (df_train.loc[~cond2, 'LifeSquare'] / df_train.loc[~cond2, 'Square']).median()

In [19]:
df_train.loc[cond2, 'LifeSquare'] = df_train['Square'] * medLife
df_test.loc[condLife(df_test), 'LifeSquare'] = df_test['Square'] * medKitchen

In [20]:
def houseFloors(df):
    df.loc[df['HouseFloor'] == 0, 'HouseFloor'] = df_train.loc[df_train['HouseFloor'] != 0, 'HouseFloor'].median()
    return df

df_train = houseFloors(df_train)
df_test = houseFloors(df_test)
## Для домов с количеством этажей = 0 ставим медиану среди всех домов в тренировочном наборе

In [21]:
df_train.loc[df_train['HouseFloor'] > df_train['HouseFloor'] , 'Floor'] = df_train['HouseFloor']
df_test.loc[df_test['HouseFloor'] > df_test['HouseFloor'] , 'Floor'] = df_test['HouseFloor']

In [22]:
df_train.describe()

,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,9984.000000,9984.000000,9984.000000,9984.000000,9984.000000,9984.000000,9.984000e+03,9984.000000,9984.000000,9984.000000,9984.000000,5196.000000,9984.000000,9984.000000,9984.000000
mean,1.888922,56.167854,34.987361,8.425735,8.524639,12.988482,3.993363e+03,0.118863,24.697716,5353.857772,8.035156,1142.809854,1.320012,4.231170,214068.636061
std,0.812961,18.844926,15.002784,4.063516,5.234671,6.436770,2.006609e+05,0.119063,17.532551,4006.029681,23.812438,1020.833557,1.493871,4.804695,92724.208665
min,1.000000,5.129222,0.370619,0.759914,1.000000,1.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,1.000000,41.784346,23.853288,6.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153907.528147
50%,2.000000,52.529412,32.581760,8.000000,7.000000,14.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192290.039215
75%,2.000000,65.890819,43.532260,10.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249100.769116
max,5.000000,198.930182,161.504222,123.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [23]:
(df_train['HouseYear']).sort_values(ascending=False)

1497    20052011
4189        4968
3485        2020
3357        2020
918         2020
          ...   
7611        1916
910         1914
2656        1912
5111        1912
9317        1910
Name: HouseYear, Length: 9984, dtype: int64

In [24]:
df_train.loc[df_train['HouseYear'] == 20052011, 'HouseYear'] = 2008 # заменим на средний

df_train.loc[df_train['HouseYear'] == 4968, 'HouseYear'] = df_train['HouseYear'].median() # непонятное значение заменим на медиану


In [25]:
(df_test['HouseYear']).sort_values(ascending=False)
## проверим тестовые данные - все ок

4389    2020
4273    2020
888     2020
3255    2020
1039    2020
        ... 
1475    1912
64      1911
2691    1910
2       1909
3253    1908
Name: HouseYear, Length: 5000, dtype: int64

In [26]:
def floorCats(df):
    df['Floor_cat'] = 0
    df.loc[df['Floor'] >= df['HouseFloor']*0.5, 'Floor_cat'] = 3
    df.loc[df['Floor'] < df['HouseFloor']*0.5, 'Floor_cat'] = 2
    df.loc[df['Floor'] == df['HouseFloor'], 'Floor_cat'] = 4
    df.loc[df['Floor'] == 1, 'Floor_cat'] = 1
    return df

df_train = floorCats(df_train)
df_test = floorCats(df_test)
# Разобьем этажи на категории, 1 - первый этаж, 2 - нижние этажи, 3 - верхние этажи, 4 - последний этаж

In [27]:
df_train.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,Floor_cat
0,14038,35,2.0,47.981561,29.442751,6.000000,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730,3
1,15053,41,3.0,65.683640,40.049543,8.000000,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063,3
2,4765,53,2.0,44.947953,29.197612,6.659216,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524,3
3,5809,58,2.0,53.352981,52.731512,9.000000,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217,2
4,10783,99,1.0,39.649192,23.776169,7.000000,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644,3


In [28]:
df_train.loc[df_train['Healthcare_1'].isna(), 'Healthcare_1'] = df_train.loc[~df_train['Healthcare_1'].isna(), 'Healthcare_1'].median()
df_test.loc[df_test['Healthcare_1'].isna(), 'Healthcare_1'] = df_test.loc[~df_test['Healthcare_1'].isna(), 'Healthcare_1'].median()

In [29]:
obj_to_num = {'A': 0, 'B': 1}
def objsToNums (df):
    df['Ecology_2'] = df['Ecology_2'].replace(obj_to_num)
    df['Ecology_3'] = df['Ecology_3'].replace(obj_to_num)
    df['Shops_2'] = df['Shops_2'].replace(obj_to_num)
    return df

df_train = objsToNums(df_train)
df_test = objsToNums(df_test)

In [30]:
# далее создадим модель и протестим ее

In [31]:
target = 'Price'

X = df_train.drop(columns=target)
y = df_train[target]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True)

In [32]:
X_train.isna().sum().sum(), X_valid.isna().sum().sum(), df_test.isna().sum().sum()

(0, 0, 0)

In [33]:
rf_model = RandomForestRegressor(criterion='mse')
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [34]:
#def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
#   print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
#    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
#    
#    plt.figure(figsize=(12,5))
#    
#    plt.subplot(121)
#    sns.scatterplot(x=train_pred_values, y=train_true_values)
#    plt.xlabel('Predicted values')
#    plt.ylabel('True values')
#    plt.title('Train sample prediction')
#    
#    plt.subplot(122)
#    sns.scatterplot(x=test_pred_values, y=test_true_values)
#    plt.xlabel('Predicted values')
#    plt.ylabel('True values')
#    plt.title('Test sample prediction')

#    plt.show()

y_train_preds = rf_model.predict(X_train)
y_test_preds = rf_model.predict(X_valid)

#evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)

In [35]:
cv_score = cross_val_score(rf_model, X_train, y_train, scoring='r2', cv=KFold(n_splits=3, shuffle=True, random_state=21))
cv_score

array([0.72880555, 0.70485864, 0.68406437])

In [36]:
feature_importances = pd.DataFrame(zip(X_train.columns, rf_model.feature_importances_), 
                                   columns=['feature_name', 'importance'])

feature_importances.sort_values(by='importance', ascending=False)

,feature_name,importance
3,Square,0.375397
13,Social_2,0.109289
12,Social_1,0.085313
2,Rooms,0.071415
14,Social_3,0.057409
1,DistrictId,0.050353
9,Ecology_1,0.035038
8,HouseYear,0.034985
4,LifeSquare,0.028194
5,KitchenSquare,0.025630


In [37]:
predictions = rf_model.predict(df_test)
predictions

array([151315.86464097, 218954.01955253, 242677.21933962, ...,
       317233.20670947, 208753.05146109, 178163.44721992])

In [38]:
submit = pd.read_csv('/kaggle/input/realestatepriceprediction/sample_submission.csv')
submit.head()
submit['Price'] = predictions
submit.head()

,Id,Price
0,725,151315.864641
1,15856,218954.019553
2,5480,242677.219340
3,15664,343985.053715
4,14275,143035.363228


In [39]:
submit.to_csv('rf_submit.csv', index=False)